In [11]:
import pandas as pd

annotator_names = ['Erik', 'Axel']
annotations = []

for name in annotator_names:
    annotations.append(pd.read_excel('annotations/Original_BIRD_Annotations.xlsx', sheet_name=name))

for annotation in annotations:
    annotation['Annotation'] = annotation['Annotation'].fillna('No description')
    print(annotation.head(3))

   Unnamed: 0            database_name table_name original_column_name  \
0           0  debit_card_specializing  yearmonth           CustomerID   
1           1  debit_card_specializing  yearmonth                 Date   
2           2  debit_card_specializing  yearmonth          Consumption   

   column_name column_description data_format value_description type  \
0  Customer ID        Customer ID     integer               NaN    F   
1          NaN               Date        text               NaN    P   
2          NaN        consumption        real               NaN  NaN   

         Annotation Comment  
0  Somewhat correct     NaN  
1  Somewhat correct     NaN  
2  Somewhat correct     NaN  
   Unnamed: 0            database_name table_name original_column_name  \
0           0  debit_card_specializing  yearmonth           CustomerID   
1           1  debit_card_specializing  yearmonth                 Date   
2           2  debit_card_specializing  yearmonth          Consumption  

In [20]:
import numpy as np
from sklearn.metrics import cohen_kappa_score

# Agreememt percentage
agreement = np.zeros((len(annotations), len(annotations)))
cohen_kappa = np.zeros((len(annotations), len(annotations)))

for anno_idx in range(len(annotations)):
    for i in range(len(annotations[anno_idx])):
        for anno_2_idx in range(len(annotations)):
            agreement[anno_idx, anno_2_idx] = np.mean(annotations[anno_idx]['Annotation'] == annotations[anno_2_idx]['Annotation'])
            cohen_kappa[anno_idx, anno_2_idx] = cohen_kappa_score(annotations[anno_idx]['Annotation'], annotations[anno_2_idx]['Annotation'])

agreement_df = pd.DataFrame(agreement, columns=annotator_names, index=annotator_names)
cohen_kappa_df = pd.DataFrame(cohen_kappa, columns=annotator_names, index=annotator_names)

print(f"The total number of annotations: {len(annotations[0])}\n")
print(f"The agreement percentage between the annotators:\n{agreement_df}\n")
print(f"The Cohen's Kappa between the annotators:\n{cohen_kappa_df}") 
# Kappa value interpretation Landis & Koch (1977):
# <0 No agreement
# 0 — .20 Slight
# .21 — .40 Fair
# .41 — .60 Moderate
# .61 — .80 Substantial
# .81–1.0 Perfect


The total number of annotations: 799

The agreement percentage between the annotators:
         Erik     Axel
Erik  1.00000  0.79224
Axel  0.79224  1.00000

The Cohen's Kappa between the annotators:
          Erik      Axel
Erik  1.000000  0.683023
Axel  0.683023  1.000000
